In [1]:
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
import json

In [2]:
df_sofiia = pd.read_csv('rdi_lonlat.csv')[["admin4Pcod", "lon", "lat"]]
df_names = pd.read_csv('archive-diploma-kaggle/all_dist_villages.csv')
df_policy = pd.read_csv('policy_planning.csv')

In [3]:
df_names = df_names[["admin4Name", "admin4Na_1", "admin4Pcod"]]

In [4]:
df_orig = df_policy.merge(df_names, left_on='pcode', right_on='admin4Pcod', how='inner', suffixes=(None, '_'))
df_orig = df_orig.merge(df_names, left_on='ref_pcode', right_on='admin4Pcod', how='inner', suffixes=(None, '_ref'))
df_orig = df_orig.merge(df_sofiia, left_on='pcode', right_on='admin4Pcod', how='inner', suffixes=(None, '_'))
df_orig = df_orig.drop(['Unnamed: 0', 'admin4Pcod', 'admin4Pcod_', 'admin4Pcod_ref'], axis=1)

df_orig = df_orig.rename(columns={"admin4Name": "name_en", "admin4Name_ref": "ref_name_en", "admin4Na_1": "name_ua", "admin4Na_1_ref": "ref_name_ua"})

In [5]:
df_orig.columns

Index(['pcode', 'cluster', 'quality', 'ref_pcode', 'ref_cluster',
       'ref_quality', 'distance', 'roads', 'ref_roads', 'diff_roads', 'cities',
       'ref_cities', 'diff_cities', 'parks', 'ref_parks', 'diff_parks',
       'banks', 'ref_banks', 'diff_banks', 'churches', 'ref_churches',
       'diff_churches', 'edu', 'ref_edu', 'diff_edu', 'elevs', 'ref_elevs',
       'diff_elevs', 'hotels', 'ref_hotels', 'diff_hotels', 'kindergartens',
       'ref_kindergartens', 'diff_kindergartens', 'libraries', 'ref_libraries',
       'diff_libraries', 'medicine', 'ref_medicine', 'diff_medicine', 'shops',
       'ref_shops', 'diff_shops', 'mobile', 'ref_mobile', 'diff_mobile',
       'post', 'ref_post', 'diff_post', 'name_en', 'name_ua', 'ref_name_en',
       'ref_name_ua', 'lon', 'lat'],
      dtype='object')

In [6]:
df_orig.head()

,pcode,cluster,quality,ref_pcode,ref_cluster,ref_quality,distance,roads,ref_roads,diff_roads,...,diff_mobile,post,ref_post,diff_post,name_en,name_ua,ref_name_en,ref_name_ua,lon,lat
0,UA2124886201,1,0.239286,UA5625886002,0,0.150794,5.000000,0,0,0,...,-1,0,0,0,Solomonovo,Соломоново,Batkiv,Батьків,22.163,48.431
1,UA2124883302,1,0.329762,UA6122481001,0,0.241270,6.244998,1,0,-1,...,0,1,0,-1,Mali Selmentsi,Малі Селменці,Velykyi Kunynets,Великий Кунинець,22.162,48.515
2,UA2124883301,1,0.299603,UA2621255301,0,0.233730,6.244998,1,0,-1,...,1,1,0,-1,Palad-Komarivtsi,Паладь-Комарівці,Slobidka Bilshivtsivska,Слобідка Більшівцівська,22.177,48.523
3,UA2124881503,1,0.292063,UA5122084801,0,0.286508,6.324555,2,1,-1,...,0,1,0,-1,Pallo,Палло,Ozerne,Озерне,22.171,48.566
4,UA2124881502,1,0.246825,UA0722183603,0,0.241270,5.567764,1,0,-1,...,0,1,0,-1,Batfa,Батфа,Voloshky,Волошки,22.189,48.569


In [7]:
def plot_borders(dst_fig):
    fig = go.Figure()

    # Load GeoJSON geometry data in X, Y format
    with open('occupation.geojson') as f:
        geojson = json.load(f)

    counter = 0
    # Loop through each feature in the GeoJSON
    for feature in geojson['features']:
        # Extract the coordinates for each polygon
        coordinates = feature['geometry']['coordinates']
        
        # If the geometry is a polygon
        if feature['geometry']['type'] == 'Polygon':
            # Extract the outer ring of the polygon
            x_coords, y_coords = zip(*coordinates[0])  # Only take the first ring
            
            # Add the polygon's boundary as lines to the figure
            fig.add_trace(go.Scattermapbox(
                fill="toself",
                lon=x_coords,
                lat=y_coords,
                mode='lines',  # Lines only
                line=dict(color='black'),  # Color of the lines
                hoverinfo='none',
                fillcolor="rgba(255, 0, 0, 0.5)",
                name = "Occupation zone",
                showlegend= True if counter == 0 else False,
                legendgroup="1"
            ))

            counter += 1

        # If the geometry is a MultiPolygon
        elif feature['geometry']['type'] == 'MultiPolygon':
            for polygon in coordinates:
                x_coords, y_coords = zip(*polygon[0])  # Take the first ring of each polygon
                
                # Add the polygon's boundary as lines to the figure
                fig.add_trace(go.Scattermapbox(
                    fill="toself",
                    lon=x_coords,
                    lat=y_coords,
                    mode='lines',  # Lines only
                    line=dict(color='black'),  # Color of the lines
                    hoverinfo='none',
                    fillcolor="rgba(255, 0, 0, 0.25)",
                    name = "Occupation zone",
                    showlegend= True if counter == 0 else False,
                    legendgroup="1"
                ))

                counter += 1

    # Remove interactive elements
    fig.update_traces(hoverinfo='none', selector=dict(type='scatter'))  # Disable hover info

    for trace in fig.data:
        dst_fig.add_trace(trace)

In [8]:
center = (0.5 * (df_orig['lon'].max() + df_orig['lon'].min()), 0.5 * (df_orig['lat'].max() + df_orig['lat'].min()))

cols = [
 'diff_roads',
 'diff_cities',
 'diff_parks',
 'diff_banks',
 'diff_churches',
 'diff_edu',
 'diff_elevs',
 'diff_hotels',
 'diff_kindergartens',
 'diff_libraries',
 'diff_medicine',
 'diff_shops',
 'diff_mobile',
 'diff_post'
]

col_label = [
    'Roads',
    'Cities',
    'Parks',
    'Banks',
    'Churches',
    'Education',
    'Elevators',
    'Hotels',
    'Kindergardens',
    'Libraries',
    'Medicine',
    'Shops',
    'Mobile',
    'Post offices'
]

colors = { 'Huge': '#d7191c', 'Major': '#f5a153', 'Moderate': '#f7ff0a', 'Minor': '#8acc62', 'None': '#1a9641' }

for i in range(len(cols)):
    column = cols[i]

    df = df_orig[(df_orig['cluster'] - df_orig['ref_cluster']) == 5 ][['lon', 'lat', "name_en", "pcode", column]]
    df[column] = np.abs(np.minimum(0, df[column]))
    df = df.sort_values(column, ascending=False)

    bins = [0, 1, 2, 4, 7, 10]
    labels = ['None', "Minor", "Moderate", "Major", "Huge"]
    df["Severity"] = pd.cut(df[column], bins=bins, labels=labels, include_lowest=True, right=False)
    df = df.rename(columns={"name_en": "Locality", "pcode": "Postal code"})

    fig = px.scatter_mapbox(df, lon='lon', lat='lat', color="Severity", 
        color_discrete_map=colors,
        hover_data={        
            'Locality': True,  # Show RDI (continuous data)
            'Postal code': True,  # Show Category
            'Severity': True
        },
        mapbox_style='open-street-map',
        zoom=5,
        center={
            'lon': center[0],
            'lat': center[1]
        },)
        # width=976,
        # height=686)

    hovertemplate = "<br>".join([
        "<b>%{customdata[0]}</b>",
        "%{customdata[1]}",
        "Severity: %{customdata[2]}<extra></extra>",
    ])

    fig.update_traces(
        marker=dict(size=6),
        hovertemplate=hovertemplate,
        # showlegend=False,
    )

    fig.update_layout(
        # title='Sample scatter plot',
        plot_bgcolor="white",
        xaxis_title=None,
        xaxis_showticklabels=False,
        yaxis_title=None,
        yaxis_showticklabels=False,
        legend=dict(
            title_text=f'Need for improvement in {col_label[i]}',
            title_font=dict(size=16, color='black'),
            x=0.1,  # Position x at the left
            y=0.1,  # Position y at the bottom
            xanchor='left',  # Anchor the x position to the left
            yanchor='bottom',  # Anchor the y position to the bottom
            orientation='v',
            bgcolor='rgba(255, 255, 255, 0.75)',  # Transparent background
            bordercolor='rgba(255, 255, 255, 0)',  # Optional: remove border
            itemsizing='constant'
        ),
    )

    fig.update_xaxes(showgrid=False, zeroline=False)
    fig.update_yaxes(showgrid=False, zeroline=False)
    fig['layout'].update(margin=dict(l=0,r=0,b=0,t=24))

    plot_borders(fig)

    config = {'scrollZoom': True}

    fig.write_html(f'policy_planning/policy_planning_{column}.html', config=config)